In [1]:
#  Day 20
#
# https://adventofcode.com/2021/day/20

from copy import *


#find min max in x and y direction of lit pixel list
def litbounds(lit):
    x0 = None
    x1 = None
    y0 = None
    y1 = None
    for x,y in lit:
        if x0 is None or x < x0:
            x0 = x
        if x1 is None or x > x1:
            x1 = x
        if y0 is None or y < y0:
            y0 = y
        if y1 is None or y > y1:
            y1 = y
    return x0,x1,y0,y1

#move lit pixels so top/left-most is 0
def trimlit(lit):
    x0,x1,y0,y1 = litbounds(lit)
    newlit = []
    for x,y in lit:
        newlit.append((x-x0, y-y0))
    
    x0,x1,y0,y1 = litbounds(lit)
    return newlit,x0,x1,y0,y1

#convert a list of list image to a list lit pixels
def litpixels(image):
    lit = []
    for y in range(len(image)):
        for x in range(len(image[0])):
            if image[y][x] == "#":
                lit.append((x,y))
    lit,x0,x1,y0,y1 = trimlit(lit)
    return lit

#parse input into a string algo and a list of lit pixels
def parse(text):
    algo, image = text.split("\n\n")
    algo = algo.replace("\n","")
    lit = litpixels([list(x) for x in image.split("\n")])
    assert(len(algo) == 512)
    #print(algo)
    return algo, lit


#create a list of list image with a boundary around lit pixels
def buildimage(lit, edgeval):
    x0,x1,y0,y1 = litbounds(lit)
    
    w = x1 + 1 + 2 
    h = y1 + 1 + 2
    
    lol = []
    for y in range(h):
        l = []
        for x in range(w):
            if x == 0 or y == 0 or x == w - 1 or y == h - 1:
                #edge condition
                l.append(edgeval)
            else:
                #inside lit area
                l.append('.')
        lol.append(l)
    
    for x,y in lit:
        lol[y+1][x+1] = '#'
    return lol
    
    

#print list of list image
def render(lit, outside):
    print("outside:", outside)
    print("\n".join(["".join(l)[:30]  for l in lit[:11]]))
    
    
def enhance(lol, algo, outside):
    w = len(lol[0]) 
    h = len(lol) 
    
    nextlol = deepcopy(lol)
    
    for y in range(h):
        for x in range(w):
            #here we need to decide color            
            b=""
            for dy in range(-1,2,1):
                for dx in range(-1,2,1):                    
                    xx = x+dx
                    yy = y+dy                    
                    if xx < 0 or xx >= w or yy < 0 or yy >= h:
                        b += outside
                    else:
                        b += lol[yy][xx]            
            b = b.replace("#", "1").replace(".","0")
            nextlol[y][x] = algo[int(b,2)]                    
            
    #Update outside value
    if outside == "#":
        outside = algo[512-1]
    else:
        outside = algo[0]
    
    return nextlol, outside
    

def solve(algo_lit, iterations, checkvalue, debug=False):
    algo, lit = algo_lit
    
    outside = '.'
    
    step = 0
    while True:
        lol = buildimage(lit, outside)    
        print("step:", step)
        if debug:            
            render(lol, outside)
            print()
        if step == iterations:
            break
        lol, outside = enhance(lol, algo, outside) 
                
        lit = litpixels(lol)
        step += 1
    
    result = len(litpixels(lol))
    
    l = "Litpixels: %d" %(result)
    
    if not checkvalue is None:
        if result == checkvalue:
            l += "   which is correct!!!"
        else:
            l += "  which should have been %s :( " % (checkvalue) 
    print(l)
    

solve(parse(open("i20_test.txt").read()), 2, 35, True)
solve(parse(open("i20.txt").read()), 2, 5275, True)

solve(parse(open("i20_test.txt").read()), 50, 3351, False)
solve(parse(open("i20.txt").read()), 50, 16482, False)

#failed guessed 5363,5155,5292,5435,5695



step: 0
outside: .
.......
.#..#..
.#.....
.##..#.
...#...
...###.
.......

step: 1
outside: .
.........
..##.##..
.#..#.#..
.##.#..#.
.####..#.
..#..##..
...##..#.
....#.#..
.........

step: 2
outside: .
...........
........#..
..#..#.#...
.#.#...###.
.#...##.#..
.#.....#.#.
..#.#####..
...#.#####.
....##.##..
.....###...
...........

Litpixels: 35   which is correct!!!
step: 0
outside: .
..............................
...##.#.#..#..##.###....######
.#.#####.#.#..#.#...##.......#
.....#..#...#...######...#.#.#
..#..#####..#..#..#.#.....#..#
.###.###...###..##.##.######..
..#.##.#.#####.....#...##.##..
.##..##.##...#######..#..#.###
.#..#......#.###.#####.##..#..
..###.....#....###..#.#...##..
...#.....#####.#.#..##..##..#.

step: 1
outside: #
##############################
####.......#..#....#..###.####
##.#.#.##.###.....##....#####.
####.#.#.###...####.....###.#.
###.###...#..#.#.#...##.##....
##.#..#.##....##.#.#.#..#.#.##
#####.#..#.#..#.##.##.#..#..#.
##.#......####.#.###.....#...